## Download counties' street networks

Using census tigerline shapefile of 2017 counties

In [1]:
import config
import geopandas as gpd
import json
import logging as lg
import networkx as nx
import os
import osmnx as ox
import pandas as pd
import time

ox.config(use_cache=True,
          log_file=True,
          log_console=True,
          log_filename='download-counties',
          cache_folder=config.counties_cache_folder)

print(ox.__version__)
print(nx.__version__)

0.8dev
2.1


In [2]:
network_type = 'drive'
retain_all = True
simplify = True

input_folder = 'input_data/counties' #tiger place shapefiles
output_folder_shapefile = config.counties_shapefile_folder #where to save graph shapefiles
output_folder_graphml = config.counties_graphml_folder #where to save graphml files

## Load shapes

In [3]:
with open('input_data/states_by_fips.json') as f:
    fips_to_state = json.load(f)

In [4]:
counties = gpd.read_file(input_folder).sort_values(by='ALAND', ascending=True)
len(counties)

3233

In [5]:
# tigerline data is epsg:4269, but osm uses epsg:4326, so project it
print(counties.crs)
counties = counties.to_crs({'init':'epsg:4326'})
counties.crs

{'init': 'epsg:4269'}


{'init': 'epsg:4326'}

In [6]:
counties[['ALAND', 'NAMELSAD', 'GEOID']].tail()

,ALAND,NAMELSAD,GEOID
1327,88635832004,Valdez-Cordova Census Area,02261
243,92326483031,Northwest Arctic Borough,02188
1509,105228855314,Bethel Census Area,02050
2246,230056014591,North Slope Borough,02185
521,377030936019,Yukon-Koyukuk Census Area,02290


## Prepare queries

In [7]:
def make_fips_state(fips):
    if fips in fips_to_state:
        return '{}_{}'.format(fips, fips_to_state[fips]['abbreviation'])

counties['fips_state'] = counties['STATEFP'].map(make_fips_state)
counties = counties.dropna(subset=['fips_state'])
len(counties)

3142

In [8]:
def make_geoid_name(row):
    return '{}_{}'.format(row['GEOID'], row['NAMELSAD']).replace(',', '').replace(' ', '_').replace('/', '_')

counties['geoid_name'] = counties.apply(make_geoid_name, axis=1)

In [9]:
queries = counties.apply(lambda row: {'fips_state':row['fips_state'],
                                      'geoid_name':row['geoid_name'],
                                      'polygon':row['geometry']}, axis=1).tolist()
queries = list(reversed(queries))
queries[0]

{'fips_state': '02_AK',
 'geoid_name': '02290_Yukon-Koyukuk_Census_Area',
 'polygon': <shapely.geometry.polygon.Polygon at 0x232ffd87198>}

## Download and save graphs

In [ ]:
start_time = time.time()

for query in queries:
    try:
        graphml_folder = '{}/{}'.format(output_folder_graphml, query['fips_state'])
        graphml_file = '{}.graphml'.format(query['geoid_name'])
        shapefile_folder = '{}/{}'.format(output_folder_shapefile, query['fips_state'])
        shapefile_file = query['geoid_name']
        
        if not os.path.exists('{}/{}'.format(graphml_folder, graphml_file)):
            
            G = ox.graph_from_polygon(polygon=query['polygon'],
                                      network_type=network_type,
                                      name=query['geoid_name'],
                                      retain_all=retain_all,
                                      simplify=simplify)
            
            ox.save_graph_shapefile(G, folder=shapefile_folder, filename=shapefile_file)
            ox.save_graphml(G, folder=graphml_folder, filename=graphml_file)
            
    except Exception as e:
        ox.log('"{}" failed: {}'.format(query['geoid_name'], e), level=lg.ERROR)
        
print('Finished making graphs in {:,.1f} seconds'.format(time.time() - start_time))

In [ ]:
# ? seconds last time